# Sample Test

In [1]:
import numpy as np

In [2]:
class Point():
    def __init__(self, x, weight = None, code = None):
        self.x = x
        self.weight = weight
        self.cluster = None
        self.code = code

class Cluster():
    def __init__(self, medoid, label):
        self.medoid = medoid
        self.label = label
        self.points = []
        self.distances = []

class KMedoids():
    def __init__(self, N, transform = None, max_iters = float('inf'), threshold_change = -float('inf'), print_iter = True):
        self.N = N # number of clusters
        self.M = [] # current medoids
        self.O = [] # current non-medoids
        self.clusters = []
        self.updated_clusters = []
        self.X = None
        self.weights = None
        self.n_points = None
        self.transform = lambda x: x
        self.max_iters = max_iters
        self.threshold_change = threshold_change
        self.num_changes = float('inf')
        self.print_iter = print_iter
        self.distsum = []

    def fit(self, X, weights, codes):
        self.__init__(self.N, self.transform, self.max_iters, self.threshold_change)
        # check data type
        if isinstance(X, np.ndarray):
            self.X = X
        else:
            self.X = np.array(X)

        self.n_points = self.X.shape[0]
        self.weights = np.array(weights)
        self.codes = np.array(codes)
        self._initialize_X()
        self._initialize_medoids()
        changed = True
        iter_num = 0
        while changed and iter_num < self.max_iters and self.num_changes > self.threshold_change:
            iter_num += 1
            if self.print_iter:
                print iter_num
            self._reset_clusters()
            self._assign_points()
            changed = self._update_medoids()
            self._create_self_O()
            self._update_costs(iter_num)
        return self.clusters

    def _initialize_X(self):
        for point, weight, code in zip(self.X, self.weights, self.codes):
            self.O.append(Point(point, weight, code))

    def _initialize_medoids(self):
        indices = np.random.randint(0, self.n_points, self.N)
        for i in indices:
            self.clusters.append(Cluster(self.O[i], i))
        self.O = np.delete(self.O, indices, axis = 0)

    def _reset_clusters(self):
        for cluster in self.clusters:
            cluster.points, cluster.distances = [], []

    def _assign_points(self):
        for point in self.O:
            
            target_distance = float('inf')
            optimal_cluster = None
            for i, cluster in enumerate(self.clusters):
                weighted_distance = self._calc_distance(point, cluster)
                if weighted_distance < target_distance:
                    target_distance = weighted_distance
                    optimal_cluster = i
            self.clusters[optimal_cluster].points.append(point)
            self.clusters[optimal_cluster].distances.append(target_distance)
            
    def _calc_distance(self, point, cluster):
        # calcs distances between point and cluster medioid, returns 
        def euclidean_distance(u, v):
            return np.sqrt(np.sum((u - v)**2))

        distance = euclidean_distance(point.x, cluster.medoid.x)
        weighted_distance = self.transform(point.weight) * distance

        return weighted_distance

    def _update_medoids(self):
        changed = False
        new_clusters = []
        total_changed = 0
        for cluster in self.clusters:
            optimal_cluster = cluster
            target_cost = sum(cluster.distances)
            for some_point in cluster.points:
                new_cluster = cluster
                new_cluster.points.append(new_cluster.medoid)
                new_cluster.points.remove(some_point)
                new_cluster.medoid = some_point
                total_cost = self._calc_cluster_cost(new_cluster.medoid, new_cluster)
                if total_cost < target_cost:
                    target_cost = total_cost
                    optimal_cluster = new_cluster
                    changed = True
                    total_changed += 1
            new_clusters.append(optimal_cluster)
        self.clusters = new_clusters
        self.num_changes = total_changed
        if not changed:
            return False
        else:
            return True
        
    def _create_self_O(self):
        self.O = []
        for cluster in self.clusters:
            self.O += cluster.points
            
    def _update_costs(self, iter_num):
        self.distsum.append((iter_num, sum(map(lambda x: sum(x.distances), self.clusters))))

    def _calc_cluster_cost(self, medoid, cluster):
        return sum(map(lambda x: self._calc_distance(x, cluster), cluster.points))

In [5]:
ncols = 2
nrows = 250
size = (ncols + 2)*nrows
data = np.random.random_integers(0, 10, size).reshape(nrows, ncols + 2)
X, code, weight = data[:,:ncols], data[:,-2], data[:,-1]

In [10]:
km = KMedoids(5, transform = lambda x: 1./x)

In [ ]:
km.fit(X, weight, code)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
